In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  %pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
%pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

     ---------------------------------------- 0.0/5.3 MB ? eta -:--:--
     ------------- -------------------------- 1.8/5.3 MB 58.8 MB/s eta 0:00:01
     ---------------------------------------  5.3/5.3 MB 67.7 MB/s eta 0:00:01
     ---------------------------------------- 5.3/5.3 MB 48.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/166.6 kB ? eta -:--:--
     ---------------------------------------- 166.6/166.6 kB ? eta 0:00:00
     ---------------------------------------- 0.0/112.8 kB ? eta -:--:--
     ---------------------------------------- 112.8/112.8 kB ? eta 0:00:00
     ---------------------------------------- 0.0/101.5 kB ? eta -:--:--
     ---------------------------------------- 101.5/101.5 kB ? eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21547 sha256=feb32d920f9823d46ab96fab54e4950d1c0af54f041f5324c144b726b6701

C:\Users\EPSY GREEN\AppData\Local\Programs\Python\Python310\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


2.17.0


In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (35) schannel: next InitializeSecurityContext failed: CRYPT_E_NO_REVOCATION_CHECK (0x80092012) - La fonction de r�vocation n�a pas pu v�rifier la r�vocation du certificat.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (35) schannel: next InitializeSecurityContext failed: CRYPT_E_NO_REVOCATION_CHECK (0x80092012) - La fonction de r�vocation n�a pas pu v�rifier la r�vocation du certificat.


In [6]:
# Load dataset
def load_data(file_path):
    df = pd.read_csv(file_path, sep='\t', header=None, names=['label', 'message'])
    df['label'] = df['label'].map({'ham': 0, 'spam': 1})  # Convert labels to 0 (ham) and 1 (spam)
    return df

In [8]:
train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"
train_data = load_data(train_file_path)
test_data = load_data(test_file_path)

In [9]:
# Feature extraction - convert the text messages into numerical features
vectorizer = CountVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(train_data['message'])
y_train = train_data['label']

In [11]:

# Additional features for training data
def extract_features(messages):
    features = pd.DataFrame()
    # Length of message
    features['length'] = messages.apply(len)
    # Count of digits
    features['digit_count'] = messages.apply(lambda x: sum(c.isdigit() for c in x))
    # Count of special characters
    features['special_char_count'] = messages.apply(lambda x: sum(c in "!@#$%^&*()-+=<>?/|\\" for c in x))
    # Count of uppercase words (often used in spam)
    features['uppercase_count'] = messages.apply(lambda x: sum(w.isupper() for w in x.split() if len(w) > 1))
    # Count of spam indicators
    spam_indicators = ["free", "win", "won", "prize", "cash", "call", "text", "urgent", "claim", "click", "offer", "limited"]
    features['spam_indicator_count'] = messages.apply(lambda x: sum(w.lower() in spam_indicators for w in x.split()))
    # Presence of phone numbers (crude check)
    features['has_phone'] = messages.apply(lambda x: 1 if sum(c.isdigit() for c in x) > 8 else 0)
    # Money symbols
    features['has_money_symbol'] = messages.apply(lambda x: 1 if any(c in x for c in "£$€") else 0)

    return features

In [12]:

# Extract additional features
train_additional_features = extract_features(train_data['message'])

# Train a Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [13]:

# Function to predict if a message is spam or ham
def predict_message(message):
    # Transform the message using the vectorizer
    message_counts = vectorizer.transform([message])

    # Extract additional features
    message_features = extract_features(pd.Series([message]))

    # Get the probability of spam
    spam_probability = clf.predict_proba(message_counts)[0][1]

    # Adjust probability based on additional features
    if message_features['has_phone'].values[0] == 1:
        spam_probability += 0.2
    if message_features['has_money_symbol'].values[0] == 1:
        spam_probability += 0.2
    if message_features['spam_indicator_count'].values[0] > 0:
        spam_probability += 0.15 * message_features['spam_indicator_count'].values[0]
    if message_features['uppercase_count'].values[0] > 1:
        spam_probability += 0.1

    # Ensure probability is between 0 and 1
    spam_probability = min(max(spam_probability, 0), 1)

    # Return the result
    label = 'spam' if spam_probability > 0.5 else 'ham'
    return [spam_probability, label]

In [14]:
# Test function (do not modify)
def test_predictions():
    test_messages = ["how are you doing today",
                    "sale today! to stop texts call 98912460324",
                    "i dont want to go. can we try it a different day? available sat",
                    "our new mobile video service is live. just install on your phone to start watching.",
                    "you have won £1000 cash! call to claim your prize.",
                    "i'll bring it tomorrow. don't forget the milk.",
                    "wow, is your arm alright. that happened to me one time too"
                   ]
    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]

    passed = True
    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        print(f"Message: {msg}")
        print(f"Prediction: {prediction[1]}, Expected: {ans}")
        print(f"Probability: {prediction[0]:.4f}")
        print("---")
        if prediction[1] != ans:
            passed = False

    if passed:
        print("You passed the challenge. Great job!")
    else:
        print("You haven't passed yet. Keep trying.")

In [15]:
test_predictions()

Message: how are you doing today
Prediction: ham, Expected: ham
Probability: 0.0015
---
Message: sale today! to stop texts call 98912460324
Prediction: spam, Expected: spam
Probability: 1.0000
---
Message: i dont want to go. can we try it a different day? available sat
Prediction: ham, Expected: ham
Probability: 0.0000
---
Message: our new mobile video service is live. just install on your phone to start watching.
Prediction: spam, Expected: spam
Probability: 0.9953
---
Message: you have won £1000 cash! call to claim your prize.
Prediction: spam, Expected: spam
Probability: 1.0000
---
Message: i'll bring it tomorrow. don't forget the milk.
Prediction: ham, Expected: ham
Probability: 0.0000
---
Message: wow, is your arm alright. that happened to me one time too
Prediction: ham, Expected: ham
Probability: 0.0003
---
You passed the challenge. Great job!


In [17]:
predict_message("message important information for o2 user. today is your lucky day! 2 find out why log onto http://www.urawinner.com there is a fantastic surprise awaiting you")

[0.9999999993431175, 'spam']